<a href="https://colab.research.google.com/github/martinab-hub/esempio_TCAV_evoluto/blob/main/esempio_TCAV_evoluto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Show cases Testing with Concept Activation Vectors (TCAV) on Imagenet Dataset and GoogleNet model**

This tutorial shows how to apply TCAV, a concept-based model interpretability algorithm, on a classification task using GoogleNet model and imagenet dataset.


In [1]:
# Install packages
!pip install captum
!pip install torch torchvision
!pip install tcav

!pip install matplotlib
!pip install Pillow
!pip install scikit-learn
!pip install scipy
!pip install tensorflow
!pip install numpy
!pip install protobuf
!pip install pandas

# Messaggio di conferma per l'installazione dei pacchetti
print("Tutti i pacchetti sono stati installati correttamente.")



Tutti i pacchetti sono stati installati correttamente.


In [2]:
import numpy as np
import os, glob

import matplotlib.pyplot as plt

import math
from pathlib import Path

import torch
from PIL import Image

from scipy.stats import ttest_ind
from sklearn.linear_model import LogisticRegression

# ..........torch imports............
import torch
import torchvision

from torch.utils.data import IterableDataset, DataLoader
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models


#.... Captum imports..................
from captum.attr import LayerGradientXActivation, LayerIntegratedGradients

from captum.concept import TCAV
from captum.concept import Concept
from captum.concept import Classifier

from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset
from captum.concept._utils.common import concepts_to_str


Defining image related transformations and functions

In [3]:
# Method to normalize an image to Imagenet mean and standard deviation
def transform(img):

    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            ),
        ]
    )(img)

In [4]:
def get_tensor_from_filename(filename):
    img = Image.open(filename).convert("RGB")
    return transform(img)


def load_image_tensors(class_name, root_path='/content/tcav_project', transform=True):
    path = os.path.join(root_path, class_name)
    filenames = glob.glob(path + '/*.jpg')

    tensors = []
    for filename in filenames:
        img = Image.open(filename).convert('RGB')
        tensors.append(transform(img) if transform else img)

    return tensors

In [5]:
def assemble_concept(name, id, concepts_path="/content/tcav_project/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    concept_iter = dataset_to_dataloader(dataset)

    return Concept(id=id, name=name, data_iter=concept_iter)

Let's assemble concepts into Concept instances using Concept class and concept images.

In this case we define five concepts: three out of five are related to image texture and patterns such as striped, zigzagged and dotted, the other two represent random concepts.

This code will:

*  Download the Broden dataset and the zebra concept from imagenet
*   Create random folders to be used by TCAV for statistical significance testing.
*   Create concept and target datasets from Imagenet and Broden and organize them in a TCAV readable structure.



In [6]:
# Clone the entire repo.
!git clone https://github.com/tensorflow/tcav.git tcav
%cd tcav
!ls

fatal: destination path 'tcav' already exists and is not an empty directory.
/content/tcav
CONTRIBUTING.md        LICENSE	  requirements.txt  Run_TCAV_on_colab.ipynb  tcav
FetchDataAndModels.sh  README.md  Run_TCAV.ipynb    setup.py


In [ ]:
# Download the dataset
%cd /content/tcav/tcav/tcav_examples/image_models/imagenet
%run download_and_make_datasets.py --source_dir='/content/tcav_project' --number_of_images_per_folder=50 --number_of_random_folders=2

/content/tcav/tcav/tcav_examples/image_models/imagenet
Downloaded 50 for zebra


In [ ]:
# Save data in drive
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

#!mkdir -p '/content/drive/MyDrive/tcav_dataset' && cp -r /content/tcav_project '/content/drive/MyDrive/tcav_dataset'

Now we have the dataset ready:

there are in total 120 images for each of the striped, zigzagged and dotted concepts;
then we randomly sample 4 diffent sets of 120 random images from imagenet dataset.

We can now associate the images to the corrisponding concept:

In [ ]:
concepts_path = "/content/tcav_project" #"data/tcav/image/concepts/"

stripes_concept = assemble_concept("striped", 0, concepts_path=concepts_path)
zigzagged_concept = assemble_concept("zigzagged", 1, concepts_path=concepts_path)
dotted_concept = assemble_concept("dotted", 2, concepts_path=concepts_path)


random_0_concept = assemble_concept("random500_0", 3, concepts_path=concepts_path)
random_1_concept = assemble_concept("random500_1", 4, concepts_path=concepts_path)

concepts = {}           # dir = {concept_name, concept_object} e.g. {'cingolo_ant': Concept(0, 'Cingolo ANT'), ...}
concepts['striped'] = stripes_concept
concepts['zigzagged'] = zigzagged_concept
concepts['dotted'] = dotted_concept


random_concepts = {}
random_concepts['random500_0'] = random_0_concept
random_concepts['random500_1'] = random_1_concept

concepts



In [ ]:
random_concepts

In [ ]:
# Let's visualize some samples from those concepts:

n_figs = 5
n_concepts = 5

fig, axs = plt.subplots(n_concepts, n_figs + 1, figsize = (25, 4 * n_concepts))

for c, concept in enumerate([stripes_concept, zigzagged_concept, dotted_concept, random_0_concept, random_1_concept]):
    concept_path = os.path.join(concepts_path, concept.name) + "/"
    img_files = glob.glob(concept_path + '*')
    for i, img_file in enumerate(img_files[:n_figs + 1]):
        if os.path.isfile(img_file):
            if i == 0:
                axs[c, i].text(1.0, 0.5, str(concept.name), ha='right', va='center', family='sans-serif', size=24)
            else:
                img = plt.imread(img_file)
                axs[c, i].imshow(img)

            axs[c, i].axis('off')

In [ ]:
# Load sample images from folder
zebra_imgs = load_image_tensors('zebra', transform=False)

Visualizing some of the images that we will use for making predictions and explaining those predictions by the means of concepts defined above.


In [ ]:
fig, axs = plt.subplots(1, 5, figsize = (25, 5))
axs[0].imshow(zebra_imgs[34])
axs[1].imshow(zebra_imgs[32])
axs[2].imshow(zebra_imgs[30])
axs[3].imshow(zebra_imgs[28])
axs[4].imshow(zebra_imgs[26])

axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
axs[3].axis('off')
axs[4].axis('off')

plt.show()

Here we perform a transformation and convert the images into tensors, so that we can use them as inputs to NN model.

In [ ]:
# Load sample images from folder
zebra_tensors = torch.stack([transform(img) for img in zebra_imgs])


**Defining GoogleNet Model**



In [ ]:
model = torchvision.models.googlenet(pretrained=True)
model = model.eval()

**Computing TCAV Scores**

Let's create TCAV class by passing the instance of GoogleNet model, a custom classifier and the list of layers where we would like to test the importance of concepts.

The custom classifier, with a default implementation of Custom Classifier, will be trained to learn classification boundaries between concepts.

In [ ]:

def get_layers_name(model: torch.nn.Module):
    """List the names of the layers in a module including nested ones.

    Args:
      model: A target PyTorch model.

    """

    layers_list = []
    for name, layer in model.named_modules():
        layers_list.append(name)
    return layers_list


def get_hidden_activation(model: torch.nn.Module,
                          layer_name,
                          input_data: torch.Tensor):
    """Get intermediate activations PyTorch model

    Args:
      model: A target PyTorch model.
      layer_name: Name of the layer of interest.
      input_data: Input passed to the PyToch model

    """

    activations = {}

    def get_hidden_activations(name):
        def hook(module, input, output):
            activations[name] = output
        return hook

    # Attach hooks to the layers whose activations you want to capture
    desired_layer = getattr(model, layer_name, None)
    if desired_layer is None:
        raise ValueError(f"Layer '{layer_name}' not found in the model.")

    hook = desired_layer.register_forward_hook(
        get_hidden_activations(layer_name))

    # Perform a forward pass to capture intermediate activations
    model(input_data)

    # Detach the hook after capturing activations
    hook.remove()

    # Access the intermediate activations from the 'activations' dictionary
    activation = activations[layer_name]

    return activation



In [ ]:
# layers = get_layers_name(model)
layers=['inception4c', 'inception4d', 'inception4e']
#layers = ['conv1', 'maxpool1', 'inception_3a', 'inception_3b', 'inception_4a', 'inception_4b', 'inception_4c', 'inception_4d', 'inception_4e', 'inception_5a', 'inception_5b', 'avgpool', 'fc']


In [ ]:
activations = {}
# Loop through the layers to extract activations and analyze them
for layer_name in layers:
    # Get activations for this layer
    activation = get_hidden_activation(model, layer_name, zebra_tensors)

    # Flatten the activations if necessary
    activations[layer_name] = activation.view(activation.size(0), -1)

activations

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Prepare the activations and labels for the classifier
activation_data = []
labels = []

for concept_name, concept in concepts.items():
    # Loop through each concept's data iterator and load activations
    for data in concept.data_iter:
        # Check the shape of 'data' before adding extra dimensions
        if len(data.shape) == 3:  # If data is [3, 224, 224]
            data = data.unsqueeze(0)  # Add batch dimension to become [1, 3, 224, 224]

        # Move data to the correct device and get hidden activation
        activation = get_hidden_activation(model, layer_name, data.to(device)).detach().cpu()

        # Flatten and add activation data
        activation_data.append(activation.view(-1).numpy())
        labels.append(concept.id)


# Convert lists to numpy arrays
activation_data = np.array(activation_data)
labels = np.array(labels)

## Train the Classifier (Logistic Regression)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(activation_data, labels)

In [ ]:
mytcav = TCAV(model=model,
              layers=layers,
              classifier = classifier,
              layer_attr_method = LayerIntegratedGradients(
                model, None, multiply_by_inputs=False))

Defining the  2 experimental sets for CAVs: ["striped", "random_0"] and ["striped", "random_1"].

We will train a classifier for each experimal set that learns hyperplanes which separate the concepts in each experimental set from one another.

In [ ]:
experimental_set_rand = {
    concept_name: [[concept, random] for random in random_concepts.values()]
    for concept_name, concept in concepts.items()
}

experimental_set_rand

In [ ]:
# zebra class index
zebra_ind = 340

scores_w_random = {}
for c in concepts:
    score_name = f'scores_{c}'
    globals()[score_name] = mytcav.interpret(inputs = zebra_tensors,
                                        experimental_sets = experimental_set_rand.get(c),
                                        target = zebra_ind,
                                        n_steps = 5,
                                       )
    scores_w_random.add[score_name] = globals()[score_name]

scores_w_random

In [ ]:
scores_zigzagged

In [ ]:
scores_dotted

In [ ]:
scores_striped

In [ ]:
# Auxiliary functions for visualizing of TCAV scores.
def format_float(f):
    return float('{:.3f}'.format(f) if abs(f) >= 0.0005 else '{:.3e}'.format(f))

def plot_tcav_scores(experimental_sets, tcav_scores):
    fig, ax = plt.subplots(1, len(experimental_sets), figsize = (25, 7))

    barWidth = 1 / (len(experimental_sets[0]) + 1)

    for idx_es, concepts in enumerate(experimental_sets):

        concepts = experimental_sets[idx_es]
        concepts_key = concepts_to_str(concepts)

        pos = [np.arange(len(layers))]
        for i in range(1, len(concepts)):
            pos.append([(x + barWidth) for x in pos[i-1]])
        _ax = (ax[idx_es] if len(experimental_sets) > 1 else ax)
        for i in range(len(concepts)):
            val = [format_float(scores['sign_count'][i]) for layer, scores in tcav_scores[concepts_key].items()]
            _ax.bar(pos[i], val, width=barWidth, edgecolor='white', label=concepts[i].name)

        # Add xticks on the middle of the group bars
        _ax.set_xlabel('Set {}'.format(str(idx_es)), fontweight='bold', fontsize=16)
        _ax.set_xticks([r + 0.3 * barWidth for r in range(len(layers))])
        _ax.set_xticklabels(layers, fontsize=16)

        # Create legend & Show graphic
        _ax.legend(fontsize=16)

    plt.show()

Let's use above defined auxilary functions and visualize tcav scores below.

In [ ]:
for c in concepts:
    score_name = f'scores_{c}'
    plot_tcav_scores(experimental_set_rand.get(c), score_name)


In [ ]:
plot_tcav_scores(experimental_set_rand.get('striped'), scores_striped)

Now, let's compute TCAV scores for a different experimental set that contains three different specific concepts such as striped, zigzagged and dotted.

In [ ]:
plot_tcav_scores(experimental_set_rand.get('dotted'), scores_dotted)

In [ ]:
plot_tcav_scores(experimental_set_rand.get('zigzagged'), scores_zigzagged)

In [ ]:
experimental_set_zig_dot = [[stripes_concept, zigzagged_concept, dotted_concept]]


In [ ]:
tcav_scores_w_zig_dot = mytcav.interpret(inputs=zebra_tensors,
                                         experimental_sets=experimental_set_zig_dot,
                                         target=zebra_ind,
                                         n_steps=5)

In [ ]:
tcav_scores_w_zig_dot

In [ ]:
plot_tcav_scores(experimental_set_zig_dot, tcav_scores_w_zig_dot)


**Statistical significance testing of concepts**

In order to convince ourselves that our concepts truly explain our predictions, we conduct statistical significance tests on TCAV scores by constructing a number of experimental sets.


Each experimental set contains a random concept consisting of a number of random subsamples. In our case this allows us to estimate the robustness of TCAV scores by the means of numerous random concepts.



In [ ]:
n = 2
random_concepts = [assemble_concept('random500_' + str(i+2), i+5) for i in range(0, n)]

print(random_concepts)

experimental_sets = [[stripes_concept, random_0_concept], [stripes_concept, random_1_concept]]

experimental_sets.extend([[stripes_concept, random_concept] for random_concept in random_concepts])
experimental_sets.append([random_0_concept, random_1_concept])
experimental_sets.extend([[random_0_concept, random_concept] for random_concept in random_concepts])
experimental_sets

Now, let's define a convenience function for assembling the experiments together as lists of Concept objects, creating and running the TCAV:

In [ ]:
def assemble_scores(scores, experimental_sets, idx, score_layer, score_type):
    score_list = []
    for concepts in experimental_sets:
        score_list.append(scores["-".join([str(c.id) for c in concepts])][score_layer][score_type][idx])

    return score_list

n addition, it is interesting to look into the p-values of statistical significance tests for each concept. We say, that we reject null hypothesis, if the p-value for concept's TCAV scores is smaller than 0.05. This indicates that the concept is important for model prediction.

We label concept populations as overlapping if p-value > 0.05 otherwise disjoint.

In [ ]:
def get_pval(scores, experimental_sets, score_layer, score_type, alpha=0.05, print_ret=False):

    P1 = assemble_scores(scores, experimental_sets, 0, score_layer, score_type)
    P2 = assemble_scores(scores, experimental_sets, 1, score_layer, score_type)

    if print_ret:
        print('P1[mean, std]: ', format_float(np.mean(P1)), format_float(np.std(P1)))
        print('P2[mean, std]: ', format_float(np.mean(P2)), format_float(np.std(P2)))

    _, pval = ttest_ind(P1, P2)

    if print_ret:
        print("p-values:", format_float(pval))

    if pval < alpha:    # alpha value is 0.05 or 5%
        relation = "Disjoint"
        if print_ret:
            print("Disjoint")
    else:
        relation = "Overlap"
        if print_ret:
            print("Overlap")

    return P1, P2, format_float(pval), relation

We now run the TCAV and obtain the scores:

In [ ]:
# Run TCAV
scores = mytcav.interpret(zebra_tensors, experimental_sets, zebra_ind, n_steps=5)

We can present the distribution of tcav scores using boxplots and the p-values indicating whether TCAV scores of those concepts are overlapping or disjoint.

In [ ]:
n = 4
def show_boxplots(layer, metric='sign_count'):

    def format_label_text(experimental_sets):
        concept_id_list = [exp.name if i == 0 else \
                             exp.name.split('_')[0] for i, exp in enumerate(experimental_sets[0])]
        return concept_id_list

    n_plots = 2

    fig, ax = plt.subplots(1, n_plots, figsize = (25, 7 * 1))
    fs = 18
    for i in range(n_plots):
        esl = experimental_sets[i * n : (i+1) * n]
        P1, P2, pval, relation = get_pval(scores, esl, layer, metric)

        ax[i].set_ylim([0, 1])
        ax[i].set_title(layer + "-" + metric + " (pval=" + str(pval) + " - " + relation + ")", fontsize=fs)
        ax[i].boxplot([P1, P2], showfliers=True)

        ax[i].set_xticklabels(format_label_text(esl), fontsize=fs)

    plt.show()

Below box plots visualize the distribution of TCAV scores for two pairs of concepts in three different layers.

In [ ]:
show_boxplots ("inception4c")
show_boxplots ("inception4d")
show_boxplots ("inception4e")
